# Create Bronze Layer Shortcuts

This notebook guides you through creating the Bronze layer shortcuts.

**Note:** Fabric shortcuts must be created via the Portal UI. This notebook provides:
1. Configuration reference
2. List of all shortcuts to create
3. Verification that shortcuts were created correctly

## Shortcuts to Create
- **24 ADLS Gen2 shortcuts** (6 dimensions + 18 facts)
- **18 Eventhouse shortcuts** (streaming events)

In [ ]:
# Configuration
BRONZE_SCHEMA = "cusn"

# ADLS Gen2 settings (update for your environment)
ADLS_ACCOUNT = "stdretail"
ADLS_CONTAINER = "supermarket"

# Eventhouse settings
EVENTHOUSE_DATABASE = "retail_eventhouse"

print(f"Bronze Schema: {BRONZE_SCHEMA}")
print(f"ADLS: https://{ADLS_ACCOUNT}.dfs.core.windows.net/{ADLS_CONTAINER}")
print(f"Eventhouse Database: {EVENTHOUSE_DATABASE}")

In [ ]:
# Create Bronze schema
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {BRONZE_SCHEMA}")
print(f"✓ Schema '{BRONZE_SCHEMA}' created")

## Step 1: Create ADLS Gen2 Shortcuts

In Fabric Portal:
1. Open your Lakehouse
2. Right-click **Tables** → **New shortcut** → **Azure Data Lake Storage Gen2**
3. Create a shortcut for each table listed below

In [ ]:
# Dimension tables (6)
dimension_tables = [
    "dim_geographies", "dim_stores", "dim_distribution_centers",
    "dim_trucks", "dim_customers", "dim_products"
]

# Fact tables (18)
fact_tables = [
    "fact_receipts", "fact_receipt_lines", "fact_store_inventory_txn",
    "fact_dc_inventory_txn", "fact_truck_moves", "fact_truck_inventory",
    "fact_foot_traffic", "fact_ble_pings", "fact_customer_zone_changes",
    "fact_marketing", "fact_online_order_headers", "fact_online_order_lines",
    "fact_payments", "fact_store_ops", "fact_stockouts",
    "fact_promotions", "fact_promo_lines", "fact_reorders"
]

all_parquet_tables = dimension_tables + fact_tables

print(f"ADLS Shortcuts to create ({len(all_parquet_tables)} total):")
print(f"\nConnection: https://{ADLS_ACCOUNT}.dfs.core.windows.net")
print(f"Container: {ADLS_CONTAINER}")
print()
for i, table in enumerate(all_parquet_tables, 1):
    print(f"  {i:2d}. {table}")

## Step 2: Create Eventhouse Shortcuts

In Fabric Portal:
1. Right-click **Tables** → **New shortcut** → **Microsoft OneLake**
2. Select your Eventhouse
3. Create a shortcut for each event table listed below

In [ ]:
# Event tables (18)
event_tables = [
    # Transaction Events
    "receipt_created", "receipt_line_added", "payment_processed",
    # Inventory Events
    "inventory_updated", "stockout_detected", "reorder_triggered",
    # Customer Events
    "customer_entered", "customer_zone_changed", "ble_ping_detected",
    # Operational Events
    "truck_arrived", "truck_departed", "store_opened", "store_closed",
    # Marketing Events
    "ad_impression", "promotion_applied",
    # Omnichannel Events
    "online_order_created", "online_order_picked", "online_order_shipped"
]

print(f"Eventhouse Shortcuts to create ({len(event_tables)} total):")
print(f"\nDatabase: {EVENTHOUSE_DATABASE}")
print()
for i, table in enumerate(event_tables, 1):
    print(f"  {i:2d}. {table}")

## Step 3: Verify Shortcuts

Run the cell below after creating all shortcuts to verify they exist.

In [ ]:
# Verify Bronze layer shortcuts
bronze_tables = spark.sql(f"SHOW TABLES IN {BRONZE_SCHEMA}").collect()

print(f"Bronze Schema ({BRONZE_SCHEMA}):")
print(f"  Total tables: {len(bronze_tables)}")
print()

# Check counts
dim_count = sum(1 for t in bronze_tables if t.tableName.startswith('dim_'))
fact_count = sum(1 for t in bronze_tables if t.tableName.startswith('fact_'))
event_count = len(bronze_tables) - dim_count - fact_count

print(f"  Dimensions: {dim_count}/6")
print(f"  Facts: {fact_count}/18")
print(f"  Events: {event_count}/18")
print()

if len(bronze_tables) >= 42:
    print("✓ Bronze layer complete!")
else:
    print(f"⚠ Missing {42 - len(bronze_tables)} shortcuts")

## Step 4: Test Data Access

Quick test to verify data is accessible.

In [ ]:
# Test reading from shortcuts
print("Testing data access...\n")

# Test dimension
try:
    df = spark.table(f"{BRONZE_SCHEMA}.dim_stores")
    print(f"✓ dim_stores: {df.count()} rows")
except Exception as e:
    print(f"✗ dim_stores: {e}")

# Test fact
try:
    df = spark.table(f"{BRONZE_SCHEMA}.fact_receipts")
    print(f"✓ fact_receipts: {df.count()} rows")
except Exception as e:
    print(f"✗ fact_receipts: {e}")

# Test event
try:
    df = spark.table(f"{BRONZE_SCHEMA}.receipt_created")
    print(f"✓ receipt_created: {df.count()} rows")
except Exception as e:
    print(f"✗ receipt_created: {e}")

print("\nBronze layer ready!")